# Linear Regression with PyTorch

Building a linear regression model using PyTorch with the California Housing dataset.

## Section 1: Import Required Libraries

Import PyTorch, scikit-learn, and other necessary libraries.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

PyTorch version: 2.9.1
NumPy version: 2.4.1
Pandas version: 2.3.3


## Section 2: Download and Load California Housing Dataset

Fetch the California Housing dataset from scikit-learn.

In [2]:
# Download the California Housing dataset
print("Downloading California Housing dataset...")
housing = fetch_california_housing()

# Extract features and target
X = housing.data
y = housing.target.reshape(-1, 1)

print(f"Dataset shape: X={X.shape}, y={y.shape}")
print(f"Features: {housing.feature_names}")


Dataset shape: X=(20640, 8), y=(20640, 1)
Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']

First few samples of X:
[[ 8.32520000e+00  4.10000000e+01  6.98412698e+00  1.02380952e+00
   3.22000000e+02  2.55555556e+00  3.78800000e+01 -1.22230000e+02]
 [ 8.30140000e+00  2.10000000e+01  6.23813708e+00  9.71880492e-01
   2.40100000e+03  2.10984183e+00  3.78600000e+01 -1.22220000e+02]
 [ 7.25740000e+00  5.20000000e+01  8.28813559e+00  1.07344633e+00
   4.96000000e+02  2.80225989e+00  3.78500000e+01 -1.22240000e+02]
 [ 5.64310000e+00  5.20000000e+01  5.81735160e+00  1.07305936e+00
   5.58000000e+02  2.54794521e+00  3.78500000e+01 -1.22250000e+02]
 [ 3.84620000e+00  5.20000000e+01  6.28185328e+00  1.08108108e+00
   5.65000000e+02  2.18146718e+00  3.78500000e+01 -1.22250000e+02]]

First few targets:
[[4.526]
 [3.585]
 [3.521]
 [3.413]
 [3.422]]


## Section 3: Preprocess and Split Data

Normalize the features and split the data into training and testing sets.

In [7]:

# Split data into training and testing sets (80-20 split) also add validation set
X_train, X_test, y_train, y_test = train_test_split(    
    X, y, test_size=0.2, random_state=42
)
# create a validation set from the training set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42
)

print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

# Convert data to PyTorch tensors
X_train=torch.FloatTensor(X_train)
# print shape of y_train before and after reshaping
print(f"y_train shape before reshaping: {y_train.shape}")
y_train=torch.FloatTensor(y_train).reshape(-1, 1)
print(f"y_train shape after reshaping: {y_train.shape}")
X_val=torch.FloatTensor(X_val)
y_val=torch.FloatTensor(y_val).reshape(-1, 1)   
X_test=torch.FloatTensor(X_test)
y_test=torch.FloatTensor(y_test).reshape(-1, 1)


means = X_train.mean(dim=0, keepdim=True)
stds = X_train.std(dim=0, keepdim=True)

X_train = (X_train - means) / stds
X_val = (X_val - means) / stds
X_test = (X_test - means) / stds




Training set size: 12384
Validation set size: 4128
Testing set size: 4128
y_train shape before reshaping: (12384, 1)
y_train shape after reshaping: torch.Size([12384, 1])


## Section 4: Build Linear Regression Model

Create a simple linear regression model using PyTorch.

In [8]:
torch.manual_seed(42)
input_features = X_train.shape[1]
w=torch.randn((input_features, 1), requires_grad=True)
b=torch.tensor(0., requires_grad=True) 


## Section 5: Training the Linear Regression Model


In [ ]:
learning_rate = 0.04
num_epochs = 1000
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Forward pass: compute predicted y
    y_pred = X_train @ w + b  # Matrix multiplication and add bias

    # Compute training loss
    loss = torch.mean((y_pred - y_train) ** 2)  # Mean Squared Error

    # Backward pass: compute gradient of the loss with respect to w and b
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

        # Zero gradients after updating
        w.grad.zero_()
        b.grad.zero_()

        # Validation loss (no gradient)
        y_val_pred = X_val @ w + b
        val_loss = torch.mean((y_val_pred - y_val) ** 2)

    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

Epoch [100/1000], Training Loss: 0.5236
Epoch [200/1000], Training Loss: 0.5184
Epoch [300/1000], Training Loss: 0.5162
Epoch [400/1000], Training Loss: 0.5152
Epoch [500/1000], Training Loss: 0.5148
Epoch [600/1000], Training Loss: 0.5146
Epoch [700/1000], Training Loss: 0.5145
Epoch [800/1000], Training Loss: 0.5144
Epoch [900/1000], Training Loss: 0.5144
Epoch [1000/1000], Training Loss: 0.5144


## Section 6: Making Predictions with the Trained Model

In [ ]:
# Sample predictions
X_new = X_test[:5]
with torch.no_grad():
    y_pred_new = X_new @ w + b
print("Predictions for first 5 samples in test set:")
print(y_pred_new)
print("Actual values for first 5 samples in test set:")
print(y_test[:5])

# Full test-set metrics
with torch.no_grad():
    y_test_pred = X_test @ w + b
    test_mse = torch.mean((y_test_pred - y_test) ** 2).item()
    test_mae = torch.mean(torch.abs(y_test_pred - y_test)).item()
    ss_res = torch.sum((y_test - y_test_pred) ** 2)
    ss_tot = torch.sum((y_test - y_test.mean()) ** 2)
    test_r2 = (1 - ss_res / ss_tot).item()

print(f"\nTest MSE: {test_mse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test R²: {test_r2:.4f}")

Predictions for first 5 samples in test set:
tensor([[0.7208],
        [1.7650],
        [2.7262],
        [2.8372],
        [2.5886]])
Actual values for first 5 samples in test set:
tensor([[0.4770],
        [0.4580],
        [5.0000],
        [2.1860],
        [2.7800]])
